## From Table 3 of Blanchard et al. 2020 (arXiv:1910.09273), we make a file for the galaxy number density distribution. --08-18-2023
Take reference from /home/zjding/csst_bao/fisher_pkmu/parameter_preparation/Sigma_nl_linear_order.ipynb

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys
sys.path.append("/home/zjding/DESI_mockchallenge/bao_fit/pymodule_barn")
from mcmc_funs import growth_factor
from scipy import interpolate, integrate

In [3]:
def Sigma2_dd_integrand(k, splPlin, R_bao, Sigma2_sm):
    Pk_lin_0 = splPlin(k)
    Sm_kernel = np.exp(-0.5*k*k * Sigma2_sm)        # 1/2 factor consistent with the convention of the smoothing factor in MC
    return Pk_lin_0*(1.0-Sm_kernel)**2.0

def cal_growth_rate(a_z, omega_m):
    a_z_p = a_z+0.01
    a_z_m = a_z-0.01
    D_p = growth_factor(1./a_z_p-1, omega_m)
    D_m = growth_factor(1./a_z_m-1, omega_m)
    f = (np.log(D_p)-np.log(D_m))/(np.log(a_z_p) - np.log(a_z_m))
    return f

In [4]:
Omega_m = 0.3075

In [5]:
zmin_array = np.array([0.9, 1.1, 1.3, 1.5])
zmax_array = np.array([1.1, 1.3, 1.5, 1.8])
dN_dOmegadz = np.array([1815.0, 1701.5, 1410.0, 940.97])
n_z = np.array([6.86e-4, 5.58e-4, 4.21e-4, 2.61e-4])
V_z = np.array([7.94e3, 9.15e3, 10.05e3, 16.22e3])
bias_array = np.array([1.46, 1.61, 1.75, 1.90])

In [6]:
zmid_array = (zmin_array + zmax_array)/2.0
a_array = 1./(1+zmid_array)
f_array = cal_growth_rate(a_array, Omega_m)
print(f_array)

[0.87308549 0.90041242 0.92083436 0.93951173]


In [7]:
# input (theoretical) linear power spectrum
ifile_pklin = "/home/zjding/DESI_mockchallenge/bao_fit/stage2_3Gpc_postrec/data/plin_model/Pk_Planck15_Table4.txt"
kwig, Plin_wig = np.loadtxt(ifile_pklin, dtype='f8', comments='#', unpack=True) 
splPlin = interpolate.InterpolatedUnivariateSpline(kwig, Plin_wig)

In [8]:
q_max = 100.0
Sigma_sm = 1.e4    # a very large number for pre-recon
G_0 = growth_factor(0.0, Omega_m)        # G_0 at z=0, normalization factor
Sigma_z_list = []
Sigma_xy_list = []
for z, f in zip(zmid_array, f_array):
    print("z, f:", z, f)
    norm_gf = growth_factor(z, Omega_m)/G_0
    const = 1.0/(6.0*np.pi**2.0) * norm_gf**2.0
    

    Sigma2_sm = Sigma_sm **2.0
    Sigma2_dd = const * integrate.quad(Sigma2_dd_integrand, kwig[0], 100.0, args=(splPlin, q_max, Sigma2_sm), epsabs=1.e-4, epsrel=1.e-4)[0]
    Sigma_xy = (2.0*Sigma2_dd)**0.5         # There is a factor of 2 due to different expression between Eq. 84 of Blanchard paper and the usual one in the BOSS analysis.
    Sigma_z = (1.0+f)*Sigma_xy
    print("Sigma_z, Sigma_xy:", Sigma_z, Sigma_xy)
    Sigma_z_list.append(Sigma_z)
    Sigma_xy_list.append(Sigma_xy)

z, f: 1.0 0.8730854886084303
Sigma_z, Sigma_xy: 9.532009751892824 5.088934706858699
z, f: 1.2000000000000002 0.9004124177444028
Sigma_z, Sigma_xy: 8.886929168555357 4.676316090958426
z, f: 1.4 0.9208343615755349
Sigma_z, Sigma_xy: 8.297942345315406 4.31996767202204
z, f: 1.65 0.9395117319777302
Sigma_z, Sigma_xy: 7.64060204302809 3.939446159078877


In [9]:
output = np.array([zmin_array, zmax_array, n_z, bias_array, f_array, np.array(Sigma_z_list), np.array(Sigma_xy_list)]).T
header = "z_low   z_up   n(z) [h/Mpc]^3   bias    growth_rate(f)     Sigma_z     Sigma_xy"
ofile = "./nz_bias_f_Sigmanl_Blanchard2020.txt"
np.savetxt(ofile, output, fmt="%.7e", header=header, comments='#')